In [ ]:
import sys
import os

import pandas as pd

# add parent directory to path
library_path = os.path.abspath('..')
if library_path not in sys.path:
    sys.path.append(library_path)

from ideal_genom.preprocessing.preparatory import Preparatory

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Create interactive widgets for input
input_path = widgets.Text(
    value='/media/luis/LaCie/valente_gwas/inputData/',
    description='Path to input PLINK binary files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

input_name = widgets.Text(
    value='/media/luis/LaCie/valente_gwas/inputData/',
    description='Prefix of PLINK binary files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

dependables_path = widgets.Text(
    value='/media/luis/LaCie/valente_gwas/dependables/',
    description='Path to dependable files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

output_path = widgets.Text(
    value='/media/luis/LaCie/valente_gwas/outputData/',
    description='Path to output files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)
output_name = widgets.Text(
    value='test_valente',
    description='Prefix of the output files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)
# Display the widgets
display(input_path, input_name, dependables_path, output_path, output_name)

# Function to get the text parameter values
def get_params():
    return input_path.value, dependables_path.value, output_path.value, output_name.value

In [ ]:
path_params = get_params()
print('input_path: ', path_params[0])
print('input_name: ', path_params[1])
print('dependables: ', path_params[2])
print('output_path: ', path_params[3])
print('output_name: ', path_params[4])

In [ ]:
preps = Preparatory(
    input_path =path_params[0],
    input_name =path_params[1],
    dependables=path_params[2],
    output_path=path_params[3],
    output_name=path_params[4]
)

In [ ]:
maf = widgets.FloatText(
    value=0.05,
    description='Minor Allele Frequency:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

geno = widgets.FloatText(
    value=0.1,
    description='Genotype missing rate:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

hwe = widgets.FloatText(
    value=5e-8,
    description='Hardy-Weinberg Equilibrium:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

mind = widgets.FloatText(
    value=0.1,
    description='Individual missing rate:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

ind_par = widgets.Textarea(
    value='50, 5, 0.2',
    description='indep pairwise (comma-separated):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='25%')
)

pca = widgets.IntText(
    value=10,
    description='Number of Principal Components:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

display(maf, geno, hwe, mind, ind_par, pca)

def get_preps_params():

    preps_params = dict()

    indep = ind_par.value.split(',')

    preps_params['maf']     = maf.value
    preps_params['geno']    = geno.value
    preps_params['hwe']     = hwe.value
    preps_params['mind']    = mind.value
    preps_params['ind_pair']= [int(indep[0]), int(indep[1]), float(indep[2])]
    preps_params['pca']     = pca.value

    return preps_params

In [ ]:
preps_params = get_preps_params()
preps_params

In [ ]:
prep_steps = {
    'ld_prune': (preps.execute_ld_prunning, {
        'maf': preps_params['maf'], 
        'geno': preps_params['geno'], 
        'mind': preps_params['mind'], 
        'hwe': preps_params['hwe'], 
        'ind_pair': preps_params['ind_pair'],
    }),
    'pca': (preps.execute_pc_decomposition, {
        'pca': preps_params['pca']
    }),
}

step_description = {
    'ld_prune': 'Linkage Disequilibrium Prunning',
    'pca'     : 'Principal Component Analysis'
}

for name, (func, params) in prep_steps.items():
    print(f"\033[1m{step_description[name]}.\033[0m")
    func(**params)
